# Day 17: Chronospatial Computer

[*Advent of Code 2024 day 17*](https://adventofcode.com/2024/day/17) and [*solution megathread*](https://redd.it/1hg38ah)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2024/17/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2024%2F17%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
part1_example_input = '''...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....'''

In [5]:
def parse_input(lines):
    nonempty_rows = set()
    nonempty_cols = set()
    galaxies = []
    for row, line in zip(range(len(lines)), lines):
        for col in range(len(line)):
            if line[col] == '#':
                nonempty_rows.add(row)
                nonempty_cols.add(col)
                galaxies.append((row, col))
    return galaxies, list(nonempty_rows), list(nonempty_cols)

parse_input(part1_example_input.splitlines())

([(0, 3), (1, 7), (2, 0), (4, 6), (5, 1), (6, 9), (8, 7), (9, 0), (9, 4)],
 [0, 1, 2, 4, 5, 6, 8, 9],
 [0, 1, 3, 4, 6, 7, 9])

In [6]:
def expand_universe(galaxies, nonempty_rows, nonempty_cols, expansion_factor=1):
    def find_empty(nonempty_sorted):
        previous = -1
        found_empty = []
        for i in nonempty_sorted:
            if i - previous > 1:
                found_empty += range(previous+1, i)
            previous = i
        return found_empty
    
    def expand_vector(nonempty_sorted, empty_sorted):
        # Here I'm not entirely happy with iterating through the entire `empty_sorted`
        # to see which are smaller, but as we only do it for rows and columns rather
        # than each galaxy, it is fine
        return [i + (expansion_factor-1)*sum(i > j for j in empty_sorted) for i in nonempty_sorted]
    
    def expand_galaxy(old_row, old_col, rowmapping, colmapping):
        return rowmapping[old_row], colmapping[old_col]
        
    empty_rows = find_empty(nonempty_rows)
    empty_cols = find_empty(nonempty_cols)
    rowmapping = dict(zip(nonempty_rows, expand_vector(nonempty_rows, empty_rows)))
    colmapping = dict(zip(nonempty_cols, expand_vector(nonempty_cols, empty_cols)))
    return [expand_galaxy(*galaxy, rowmapping, colmapping) for galaxy in galaxies]

expand_universe(*parse_input(part1_example_input.splitlines()))

[(0, 3), (1, 7), (2, 0), (4, 6), (5, 1), (6, 9), (8, 7), (9, 0), (9, 4)]

In [7]:
def galaxies_distances(galaxy, galaxies):
    # print(f"{galaxy=}, {galaxies=}")
    if not galaxy:
        return galaxies_distances(galaxies[0], galaxies[1:])
    head = {f"{galaxy} - {row, col}": abs(row - galaxy[0]) + abs(col - galaxy[1]) for row, col in galaxies}
    if galaxies:
        head.update(galaxies_distances(galaxies[0], galaxies[1:]))
    return head

sum(galaxies_distances(None, expand_universe(*parse_input(part1_example_input.splitlines()))).values())
# sum(galaxies_distances(None, expand_universe(*parse_input(downloaded['input'].splitlines()))).values())

292

In [4]:
# HTML(downloaded['part2'])

In [9]:
print(sum(galaxies_distances(None, expand_universe(*parse_input(part1_example_input.splitlines()), expansion_factor=10)).values()))
print(sum(galaxies_distances(None, expand_universe(*parse_input(part1_example_input.splitlines()), expansion_factor=100)).values()))
# sum(galaxies_distances(None, expand_universe(*parse_input(downloaded['input'].splitlines()), expansion_factor=1000000)).values())

1030
8410
